In [3]:
import os

In [4]:
%pwd

'd:\\Nikku\\End to End Project\\Wine-Quality-Tester\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Nikku\\End to End Project\\Wine-Quality-Tester'

In [7]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/itzmeakashps/Wine-Quality-Tester.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "itzmeakashps"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "d587244e64e62ea9ab9a724d252a21ca83c524fe"

In [17]:
# Prepare  Entity

from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class ModelEavluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_url: str

In [18]:
# Prepare Configuration Manager

from winetester.constant import *
from winetester.utils.common import read_yaml, create_directories



class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEavluationConfig:
        config  = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEavluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_url = "https://dagshub.com/itzmeakashps/Wine-Quality-Tester.mlflow"
        )
        
        return model_evaluation_config

In [19]:
# Prepare Components

import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from winetester.utils.common import save_json




In [20]:
class ModelEvaluation:
    def __init__(self, config:ModelEavluationConfig):
        self.config = config
        
    def eval_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metric(test_y, predicted_qualities)
            
            # Saving metrics as local
            
            scores = {"rmse" : rmse, "mae": mae, "r2" : r2}
            save_json(path=Path(self.config.metric_file_name), data = scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            
            # Model registry does not work with file store
            if tracking_url_type_store  != "file":
                
                
                # Register the model
                # There are other ways to use the Model Registry, Which depends on the use case,
                # Please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
                
                

In [21]:
# Prepare the pipeline

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-10-10 10:56:59,345: INFO: Yaml file  : config\config.yaml loadded successfully]
[2023-10-10 10:56:59,346: INFO: Yaml file  : params.yaml loadded successfully]
[2023-10-10 10:56:59,347: INFO: Yaml file  : schema.yaml loadded successfully]
[2023-10-10 10:56:59,348: INFO: created directory at : artifacts]
[2023-10-10 10:56:59,348: INFO: created directory at : artifacts/model_evaluation]
[2023-10-10 10:57:01,918: INFO: json file is saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/10/10 10:58:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
